# Brain data

This section presents results of brain MRI data. Below are quantitative T1 values computed using the MP2RAGE and the MTsat methods. These values are averaged within the gray matter and white matter masks.

### Code imports

In [ ]:
# Python imports 
from IPython.display import clear_output
from pathlib import Path
import numpy as np

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 1)

# Import custom tools
from tools.data import Data
from tools.plot import Plot
from tools.stats import Stats

### Download data

In [ ]:
data_type = 'brain'
release_version = 'latest'

dataset = Data(data_type)
dataset.download(release_version)

### Load data plot it

### qMRI Metrics

In [ ]:
dataset.load()
fig_gm = Plot(dataset, plot_name = 'brain-1')

fig_gm.title = 'Brain qMRI microstructure measures'



In [ ]:
# If you're running this notebook in a Jupyter Notebook (eg, on MyBinder), change 'jupyter-book' to 'notebook'
fig_gm.display('jupyter-book')


#### Statistics

##### White Matter

In [ ]:
stats_wm = Stats(dataset)
stats_wm.build_df('WM')
stats_wm.build_stats_table()
display(stats_wm.stats_table)

##### Grey Matter

In [ ]:
stats_gm = Stats(dataset)
stats_gm.build_df('GM')
stats_gm.build_stats_table()
display(stats_gm.stats_table)

### Diffusion Tracts

In [ ]:
data_type = 'brain-diffusion'
release_version = 'latest'

dataset = Data(data_type)
dataset.download(release_version)

In [ ]:
dataset.load()

fig_diff = Plot(dataset, plot_name = 'brain-diff')

fig_diff.title = 'Brain qMRI diffusion measures'

fig_diff.display('jupyter-book')

#### Statistics

In [ ]:
stats_cc1 = Stats(dataset)
stats_cc1.build_df('CC_1')
stats_cc1.build_stats_table()
display(stats_cc1.stats_table)

In [ ]:
stats_mcp = Stats(dataset)
stats_mcp.build_df('MCP')
stats_mcp.build_stats_table()
display(stats_mcp.stats_table)

### Diffusion - Corpus Callosum

In [ ]:
data_type = 'brain-diffusion-cc'
release_version = 'latest'

dataset = Data(data_type)
dataset.download(release_version)

In [ ]:
dataset.load()

fig_diff = Plot(dataset, plot_name = 'brain-diff-cc')

fig_diff.title = 'Brain qMRI diffusion measures - corpus callosum'

fig_diff.display('jupyter-book')

#### Statistics

##### Genu

In [ ]:
stats_cc1 = Stats(dataset)
stats_cc1.build_df('genu')
stats_cc1.build_stats_table()
display(stats_cc1.stats_table)

##### Body

In [ ]:
stats_cc1 = Stats(dataset)
stats_cc1.build_df('body')
stats_cc1.build_stats_table()
display(stats_cc1.stats_table)

##### Splenium

In [ ]:
stats_cc1 = Stats(dataset)
stats_cc1.build_df('splenium')
stats_cc1.build_stats_table()
display(stats_cc1.stats_table)